In [3]:
#!/home/data/liuchang/anaconda3/envs/py3/bin/python
# -*- coding:utf-8 -*-
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import os 
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')# 忽略警告
import logging
import os.path
import sys
import multiprocessing
import gensim    
import torch
from torch.utils.data import *
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd
# xlist = np.load('/home/data/liuchang/data/wikiw2v.npy') 
# ylist = np.load('/home/data/liuchang/data/wikipunc.npy') 
# del xlist
# del ylist
# gc.collect()

# dataloader
# batch_size = 1
# class wikiset(Dataset):
#     def __init__(self):
#         print('dataset loading...')
#         self.xlist = np.load('/home/data/liuchang/data/wikiw2v.npy') 
#         self.ylist = np.load('/home/data/liuchang/data/wikipunc.npy') 
        
#     def __len__(self):
#         return len(self.ylist)
#     def __getitem__(self,idx):
#         return torch.FloatTensor(self.xlist[idx]),torch.FloatTensor(self.ylist[idx])

# teset = wikiset()        
# seqloader = DataLoader(teset,batch_size=1,shuffle = False ,num_workers = 4)            

class TLSTM(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(TLSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(input_dim,hidden_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return torch.zeros(1,1,self.hidden_dim),torch.zeros(1,1,self.hidden_dim)

    def forward(self,inputs):
        lstm_out, self.hidden = self.lstm(inputs.view(len(inputs),1,-1),self.hidden)
        tag_space = self.fc(lstm_out.view(len(inputs),-1))
        tag_scores = F.softmax(tag_space,dim = 1)
        return tag_scores
lstm = TLSTM(200,100,6)
model = gensim.models.Word2Vec.load('/home/data/liuchang/data/Tinywikizh.model')
lstm.load_state_dict(torch.load('tlstm.pkl'))


In [4]:
def print_punc(sentence):
    senseg = ' '.join(jieba.cut(sentence,cut_all=False))
    print(senseg)
    zerovec = np.zeros(200)
    xpara = []
    for word in senseg.split():
        try:
            xpara.append(model[word])
        except:
            xpara.append(zerovec)
            # torch.load(lstm.state_dict(), 'tlstm.pkl')
    x = torch.FloatTensor(xpara)
    with torch.no_grad():
        lstm.hidden = lstm.init_hidden()
        tag_scores = lstm(x)
    # y = lstm(xpara[0])
    print('raw sentence:',sentence)
    a,index = torch.max(tag_scores,dim = 1) 
    print('punc index:',index)
    punc_dict = {0:'',1:'，',2:'。',3:'！',4:'？',5:'、'}
    word = senseg.split()
    puncsen = ''
    for i in range(0,len(tag_scores)):
        puncsen += word[i]+punc_dict[int(index[i])]
    print('cyberpuncor result:',puncsen)


In [6]:
import time
import jieba
tic = time.time()
print_punc('基于长短期记忆神经网络的标点恢复模型是真他妈的好用啊')
print(time.time()-tic)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.185 seconds.
Prefix dict has been built succesfully.
/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


基于 长短期 记忆 神经网络 的 标点 恢复 模型 是 真 他妈的 好用 啊
raw sentence: 基于长短期记忆神经网络的标点恢复模型是真他妈的好用啊
punc index: tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3])
cyberpuncor result: 基于长短期记忆神经网络的标点，恢复模型是真他妈的好用啊！
3.95646333694458


基于 长短期 记忆 神经网络 的 标点 恢复 模型 是 真 他妈的 好用 啊


/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


raw sentence: 基于长短期记忆神经网络的标点恢复模型是真他妈的好用啊
punc index: tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3])
cyberpuncor result: 基于长短期记忆神经网络的标点恢复模型，是真他妈的好用啊！
1.9918181896209717


In [ ]:
# s = '哈哈哈哈我现在在找各种各样乱七八糟的文本丢到这个破网络里看看结果咋样'
print(len(s))